In [1]:
import numpy as np
import scipy as sp
import cv2
import os
import glob
import pickle
import MTCNN as mtcnn
import GenFeature as genfeat
import scipy
import time
from time import sleep
feature_file = np.load("feature_file.npz")
UNKNOWN_LABEL = "Unknown"
THRESHOLD = 0.5

In [2]:
def draw_label(image, point, label, emotion, font=cv2.FONT_HERSHEY_SIMPLEX,
                   font_scale=1, thickness=2):
        size = cv2.getTextSize(label, font, font_scale, thickness)[0]
        x, y = point
        cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), -1)
        cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)
        cv2.putText(image, emotion, (point[0], point[1]+20), font, font_scale, (255, 255, 255), thickness)

label_array = []
feature_array = []
for key in feature_file:
    label_array.append(key)
    feature_array.append(feature_file[key])

def get_label(feature):
    distance = float("inf")
    label = None

    dist = scipy.spatial.distance.cdist(feature.reshape((1, feature.size)), np.array(feature_array), 'cosine')
    closest_index = np.argmin(dist)
    distance, label = dist[0][closest_index], label_array[closest_index] 

    return label if distance < THRESHOLD else UNKNOWN_LABEL 


def get_margins(face_margin, margin=1):
    (x, y, w, h) = face_margin[0], face_margin[1], face_margin[2] - face_margin[0], face_margin[3] - face_margin[1]
    margin = int(min(w, h) * margin / 100)
    x_a = int(x - margin)
    y_a = int(y - margin)
    x_b = int(x + w + margin)
    y_b = int(y + h + margin)
    return (x_a, y_a, x_b - x_a, y_b - y_a)


def face_recon(video_file):
    #video_capture = cv2.VideoCapture('20190610_221401.avi')
    video_capture = cv2.VideoCapture(video_file)
    while (video_capture.isOpened()):
        if not video_capture.isOpened():
            sleep(5)
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        if (ret != True):
              break
        #if (i == 14):
        original_frame = frame.copy()
        _, boundingboxes, features, emotion = mtcnn.process_image(frame)

        # placeholder for cropped faces
        for i in range(boundingboxes.shape[0]):
            (x, y, w, h) = get_margins(boundingboxes[i, 0:4])
            label = get_label(features[i]) if i < len(features) else UNKNOWN_LABEL
            #print len(features), i
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 200, 0), 2)
            #label = "{}".format(label)
            #draw_label(frame, (x,y), "{}{}".format(label, emotion))
            draw_label(frame, (x,y), label, emotion)
    
        cv2.imshow('Face Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # ESC key press
            break
    # When everything is done, release the capture
    video_capture.release()
    cv2.destroyAllWindows()
    return label, original_frame


#if __name__ == "__main__":
    #face_recon()


In [3]:
label,image = face_recon('video1.mp4')

In [ ]:
label

## If label is "Unknown" type the desired name in the variable "userName"

In [ ]:

userName = "taylor"

## Save the  image with desired name and generate features and store them

In [ ]:
if ((label == "Unknown")):
    cv2.imwrite("/home/jug.971990/Ramakrishna/data_collection/userdata/userimages/"+userName+".jpg", image)
    genfeat.generate_features()